# Capstone Project: Building an Interactive AI Rainbow on a Budget Chatbot

We perform the backend processing for the chatbot.

![Retrieval Augmented Generation(RAG)](https://miro.medium.com/v2/resize:fit:720/format:webp/1*UyhiO87T-hejRhqI7EwvgA.png)


## Contents
1. [Import libraries, API and set filepath](#Import-libraries,-API-and-set-filepath)
2. [Load the data](#Load-the-data)
3. [Build Index](#Build-Index)
4. [Train Generation](#Train-Generation)
5. [Eval Generation](#Eval-Generation)
6. [Initial Evaluation](#Initial-Evaluation)
7. [2nd Evaluation](#2nd-Evaluation)
8. [Exploring Differences Between The 2 Different Models](#Exploring-Differences-Between-The-2-Different-Models)
9. [Checking whether the GPT-3.5-turbo-1106 (Fine Tune) or GPT-4-1106-preview works better](#Checking-whether-the-GPT-3.5-turbo-1106-(Fine-Tune)-or-GPT-4-1106-preview-works-better)

# AI Rainbow on a Budget Chatbot


To study the manual scrap datasets "products.csv"

## Import libraries, API and set filepath

In [3]:
import os

from llama_index import Document, GPTVectorStoreIndex, ServiceContext
from llama_index.readers import BeautifulSoupWebReader, SimpleDirectoryReader
from llama_index.llms import OpenAI
from llama_index.evaluation import DatasetGenerator

import openai

In [5]:
# set filepath to my data directory 

current_dir = os.getcwd()
data_dir = os.path.join(current_dir, "./data")


## Load the data

In [6]:
from pathlib import Path 
from llama_index import download_loader

PagedCSVReader = download_loader("PagedCSVReader")

loader = PagedCSVReader(encoding="utf-8")
docs = loader.load_data(file=Path('./data/products.csv')) 

#read and load the csv file into the model

## Build Index

With all the data loaded, we can construct the index for the chatbot. There are 4 types of indexing: Summary index, VectorStore Index, Tree Index and Keyword Table Index. Here we are using VectorStore Index, which is also one of the most common types of indexing.

In [7]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

service_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4-turbo", temperature=0) # degree of randomness from 0 to 1. 
)

index = GPTVectorStoreIndex.from_documents(documents=docs, service_context=service_context)


#method is used to create an index from a set of documents (docs).

In [8]:
# saving the output as a vector store so that we can refer to this 
# instead of running the embedding model above again

index.storage_context.persist(persist_dir="./data/index.vecstore")  

#the data is stored in that location, making it easier to load the index from storage later on without having to re-index the documents. 


1. Generate a training and evaluation dataset using gpt-3.5-turbo
2. Use gpt-4 on the evaluation questions to get our first model performance
3. Generate another set of training dataset using GPT-3.5-turbo
4. Use gpt-3.5-turbo-1106 on the evaluation question to get our second model performance
5. Generate another set of training dataset using GPT-3.5-turbo
6. Use gpt-4-1106-preview on the evaluation question to get our third model performance
7. Explore the three different models and see whether are there any differences

## Train Generation

In [9]:
# Shuffle the documents
import random

random.seed(42)
random.shuffle(docs)

gpt_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4-turbo", temperature=0)
)

In [1]:
question_gen_query=(
    "Craft a series of questions that potential customers might ask about what other fruits and vegetables to add to their diet to achieve the optimal nutrition. The 5-colour diet includes fruits and vegetables from the red, yellow/orange, white/tan/brown, green, blue/purple/black colour categories. The data file includes fruits and vegetables from these 5 colours as columns. Check what vegetables and fruits colour their cart already has and then suggest fruits and vegetables of colours that are missing from their cart by checking for the best match with the row of data in the redProductName, yellowProductName, whiteProductName, greenProductName, blueProductName fields. Once that is done, suggest the respective recipe under the recipes column."
)
# find out more about question generation from 
# https://gpt-index.readthedocs.io/en/latest/examples/evaluation/QuestionGeneration.html

dataset_generator = DatasetGenerator.from_documents(
    docs[:24],
    question_gen_query=question_gen_query,
    service_context=gpt_context,
)

NameError: name 'DatasetGenerator' is not defined

In [11]:
import nest_asyncio
nest_asyncio.apply()

#The code snippet import nest_asyncio; nest_asyncio.apply() is used to enable nested event loops in asyncio, which is the default behavior in Python's asyncio library.

In [12]:
questions = dataset_generator.generate_questions_from_nodes(num=25)   
print("Generated ", len(questions), " questions")
#generates a list of questions based on the documents. 25 questions generated

Generated  25  questions


In [13]:
with open("train_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")
        print(question)
        
# writes a list of questions to a file named train_questions.txt and prints each question to the console.

How can the placement of my stove impact the Feng Shui of my kitchen and overall living space?
What are the recommended directions for placing an aquarium in order to enhance positive energy flow in my home?
How does the quality of object placement affect the Feng Shui of a room or space?
Can you provide guidance on the optimal direction to place my bed for better Feng Shui in my bedroom?
What are some common mistakes to avoid when it comes to the placement of objects in accordance with Feng Shui principles?
How can conflicting elements like fire, water, and earth impact the Feng Shui of a space, particularly when it comes to placing an aquarium?
Are there specific areas in my home where I should avoid placing certain objects, such as an aquarium, to maintain good Feng Shui?
How does the scale of an object impact its placement in relation to Feng Shui principles?
Can you explain the significance of the North-West direction in Feng Shui and how it can influence the energy flow in a livi

How can the placement of my stove impact the Feng Shui of my kitchen and overall living space?
What are the recommended directions for placing an aquarium in order to enhance positive energy flow in my home?
How does the quality of object placement affect the Feng Shui of a room or space?
Can you provide guidance on the optimal direction to place my bed for better Feng Shui in my bedroom?
What are some common mistakes to avoid when it comes to the placement of objects in accordance with Feng Shui principles?
How can conflicting elements like fire, water, and earth impact the Feng Shui of a space, particularly when it comes to placing an aquarium?
Are there specific areas in my home where I should avoid placing certain objects, such as an aquarium, to maintain good Feng Shui?
How does the scale of an object impact its placement in relation to Feng Shui principles?
Can you explain the significance of the North-West direction in Feng Shui and how it can influence the energy flow in a living space?
What additional tips should I keep in mind when redesigning my living space to ensure that it aligns with the principles of Feng Shui for optimal balance and harmony?
How does the direction in which my stove is placed impact the Feng Shui of my kitchen and overall living space?
What are the recommended guidelines for positioning an aquarium in a way that enhances the energy flow and promotes prosperity in my home?
Can you explain the significance of placing the bed headroom facing South-West and how it affects the energy flow in the bedroom?
What are the potential consequences of having a bed headroom with bad placement in terms of Feng Shui principles?
How can I ensure that there is a solid wall behind my headboard to support the bed and maintain stable energy flow around it?
What does it mean to have a commanding position for the bed in Feng Shui, and how can I achieve this in my bedroom layout?
How does the visibility of the bedroom door from the bed impact the sense of security and overall energy flow in the room according to Feng Shui principles?
Are there any specific recommendations for the placement of other objects in my living space, such as mirrors or plants, to enhance the Feng Shui and promote positive energy flow?
How can I incorporate Feng Shui principles into the overall design and layout of my living space to create a harmonious and balanced environment?
What are some common mistakes to avoid when applying Feng Shui principles to the redesign of my living space, and how can I ensure that I am following the guidelines correctly for optimal results?
How can the placement of my stove impact the Feng Shui of my kitchen and overall living space?
What are the recommended directions for placing an aquarium to enhance the flow of positive energy in my home?
How does the direction in which my bed is positioned affect my sleep quality and overall well-being according to Feng Shui principles?
Can you provide guidance on the optimal placement of a safe box to attract wealth and prosperity into my home?
What Feng Shui wealth items should I consider incorporating into my safe box to enhance its energy and attract abundance?

In [14]:
input_file_path = 'train_questions.txt'
output_file_path = 'modified_train_questions.txt'

def postprocess(input_file_path, output_file_path):
    with open(input_file_path, 'r') as file:
        modified_lines = [line.replace("Question:", "").strip() for line in file]

    with open(output_file_path, 'w') as new_file:
        for line in modified_lines:
            new_file.write(line + '\n')
            
#modifies the content by removing a specific string ("Question:") from each line, and writes the modified lines to a new output file 

## Eval Generation

In [15]:
dataset_generator = DatasetGenerator.from_documents(
    docs[
        25:49
    ],
    question_gen_query=question_gen_query,
    service_context=gpt_context,
)

# generate datasets (potentially question-answer pairs) from a specified subset of documents. The subset is defined by slicing the docs list from index 30 to 54 (Python slicing is exclusive of the end index).

In [16]:
questions = dataset_generator.generate_questions_from_nodes(num=25)
print("Generated ", len(questions), " questions")
#generates a list of questions based on the documents. 25 question generated

Generated  25  questions


In [17]:
with open("eval_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")
#writes a list of questions to a file named eval_questions.txt

In [18]:
input_file_path = 'eval_questions.txt'
output_file_path = 'modified_eval_questions.txt'

postprocess(input_file_path, output_file_path)

#modify the content of a file named eval_questions.txt and write the modified content to a new file named modified_eval_questions.txt. 

In [19]:
print("Total number of documents:", len(docs))
#prints the total number of documents in the docs list.

Total number of documents: 52


## Initial Evaluation

For this evaluation with GPT-4 Query Engine, we will be using the [`ragas` evaluation library](https://github.com/explodinggradients/ragas).

For this notebook, we will be using the following two metrics:

- `answer_relevancy` - This measures how relevant is the generated answer to the prompt. If the generated answer is incomplete or contains redundant information the score will be low. This is quantified by working out the chance of an LLM generating the given question using the generated answer. Values range (0,1), higher the better.  
- `faithfulness` - This measures the factual consistency of the generated answer against the given context. This is done using a multi step paradigm that includes creation of statements from the generated answer followed by verifying each of these statements against the context. The answer is scaled to (0,1) range. Higher the better.

In [21]:
questions = []
with open("modified_eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())
#reads questions from a file named modified_eval_questions.txt and stores them in a list named questions.

In [22]:
from llama_index import VectorStoreIndex

# limit the context window to 2048 tokens so that refine is used
gpt_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4", temperature=0), context_window=2048
)

index = VectorStoreIndex.from_documents(docs, service_context=gpt_context)

query_engine = index.as_query_engine(similarity_top_k=2)

#GPT-4 model to understand the semantic content of documents and then uses this understanding to find documents that are semantically similar to a given query. 

In [23]:
contexts = []
answers = []


for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

#store the contexts and answers of the responses

In [250]:
questions[:25]

['How can the placement of objects like a stove, aquarium, and bed impact the energy flow in my living space according to Feng Shui principles?',
 'What are the recommended directions for placing a stove, aquarium, and bed to promote positive energy and well-being in my home?',
 'How does the direction in which objects are placed affect the Feng Shui of a space, and what specific benefits can be achieved by following these guidelines?',
 'Can you provide examples of how improper placement of objects can create negative energy in a living space, and how adjusting their direction can improve the overall Feng Shui?',
 'What are some key considerations to keep in mind when selecting and positioning plants in my home to enhance the positive energy flow according to Feng Shui principles?',
 'How can I ensure that the plants I choose for my living space are healthy and thriving, and what impact does the health of plants have on the energy within a home?',
 'In what ways can the placement of p

In [251]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)


result = evaluate(ds,[answer_relevancy, faithfulness])
print(result)

# Evaluate the answer_relevancy & faithfulness using ragas. Ragas score  = (answer_relevancy + faithfulness) / 2

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

{'answer_relevancy': 0.8894, 'faithfulness': 0.9822}


{'ragas_score': 0.9358, 'answer_relevancy': 0.8894, 'faithfulness': 0.9822}

## 2nd Evaluation

For this evaluation with GPT-4-1106-preview Query Engine, we will be using the [`ragas` evaluation library](https://github.com/explodinggradients/ragas).

For this notebook, we will be using the following two metrics:

- `answer_relevancy` - This measures how relevant is the generated answer to the prompt. If the generated answer is incomplete or contains redundant information the score will be low. This is quantified by working out the chance of an LLM generating the given question using the generated answer. Values range (0,1), higher the better.  
- `faithfulness` - This measures the factual consistency of the generated answer against the given context. This is done using a multi step paradigm that includes creation of statements from the generated answer followed by verifying each of these statements against the context. The answer is scaled to (0,1) range. Higher the better.

In [257]:
questions = []
with open("modified_eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())
#reads questions from a file named modified_eval_questions.txt and stores them in a list named questions

In [258]:
from llama_index import VectorStoreIndex

# limit the context window to 2048 tokens so that refine is used
gpt_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4-1106-preview", temperature=0), context_window=2048
)

index = VectorStoreIndex.from_documents(docs, service_context=gpt_context)

query_engine = index.as_query_engine(similarity_top_k=2)
#querying documents based on their semantic similarity to a given query.

In [ ]:
contexts = []
answers = []


for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))
    
#store the contexts and answers of the responses.

In [260]:
questions[:25]
#create a new list containing the first 25 elements from the questions list.

['How can the placement of objects like a stove, aquarium, and bed impact the energy flow in my living space according to Feng Shui principles?',
 'What are the recommended directions for placing a stove, aquarium, and bed to promote positive energy and well-being in my home?',
 'How does the direction in which objects are placed affect the Feng Shui of a space, and what specific benefits can be achieved by following these guidelines?',
 'Can you provide examples of how improper placement of objects can create negative energy in a living space, and how adjusting their direction can improve the overall Feng Shui?',
 'What are some key considerations to keep in mind when selecting and positioning plants in my home to enhance the positive energy flow according to Feng Shui principles?',
 'How can I ensure that the plants I choose for my living space are healthy and thriving, and what impact does the health of plants have on the energy within a home?',
 'In what ways can the placement of p

In [261]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)


result = evaluate(ds,[answer_relevancy, faithfulness])
print(result)

#evaluate result on answer_relevancy & faithfulness using ragas. Ragas score  = (answer_relevancy + faithfulness) / 2


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

{'answer_relevancy': 0.9329, 'faithfulness': 0.9822}


{'ragas_score': 0.9575, 'answer_relevancy': 0.9329, 'faithfulness': 0.9822}

| Model                | Ragas Score | Answer Relevancy   | Faithfulness   |
|:----------------:    |:-----------:|:------------------:|:--------------:|
| GPT-4                | 0.9358      | 0.8894             | 0.9822         |
| GPT-4.0-1106-preview | 0.9575      | 0.9266             | 0.9840         |
{'ragas_score': 0.9358, 'answer_relevancy': 0.8894, 'faithfulness': 0.9822}
{'ragas_score': 0.9575, 'answer_relevancy': 0.9329, 'faithfulness': 0.9822}

Date : 27 April 2024

## Exploring Differences Between The 2 Different Models

Let's quickly compare the differences in responses, to demonstrate that fine tuning did indeed change something.

In [35]:
index = VectorStoreIndex.from_documents(docs)
#create an index from a collection of documents 

In [36]:
questions = []
with open("modified_eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())
#reads questions from a file named modified_eval_questions.txt and stores them in a list named questions

In [37]:
print(questions[1])  
#access the second index/element in the questions

What are the recommended directions for placing a stove, aquarium, and bed to promote positive energy and well-being in my home?


What are the recommended directions for placing a stove, aquarium, and bed to promote positive energy and well-being in my home?


## GPT-4

In [54]:
from llama_index.response.notebook_utils import display_response

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4", temperature=0),
    context_window=2048,  # limit the context window artifically to test refine process
)

#preparing the environment for querying or indexing documents using the LlamaIndex library. 

In [55]:
query_engine = index.as_query_engine(service_context=gpt_35_context)

response = query_engine.query(questions[1])

display_response(response)
#query an index with a specific service context and display the response

**`Final Response:`** The context only provides information about the placement of an aquarium. It is recommended to place an aquarium in the North-West or South-East direction of your home. However, you should avoid placing it in the kitchen, bedroom, bathroom, or the center of the home due to the conflicting elements of fire, water, and earth. Unfortunately, the context does not provide information about the recommended directions for a stove or a bed.

Final Response: The context only provides information about the placement of an aquarium. It is recommended to place an aquarium in the North-West or South-East direction of your home. However, you should avoid placing it in the kitchen, bedroom, bathroom, or the center of the home due to the conflicting elements of fire, water, and earth. Unfortunately, the context does not provide information about the recommended directions for a stove or a bed.



## GPT-4.0-1106-preview

In [58]:
ft_context = ServiceContext.from_defaults(
    #llm=ft_llm,
    llm=OpenAI(model="gpt-4-1106-preview",temperature=0),
    context_window=2048,  # limit the context window artifically to test refine process
)
#environment for querying or indexing documents using the LlamaIndex library

In [59]:
query_engine = index.as_query_engine(service_context=ft_context)

response = query_engine.query(questions[1])

display_response(response)
#query an index with a specific service context and display the response

**`Final Response:`** The recommended directions for placing an aquarium to promote positive energy and well-being in your home are North-West and South-East. It is important to note that you should avoid placing an aquarium in the kitchen, bedroom, bathroom, or the center of the home, as these locations are considered inauspicious due to the conflicting elements of fire, water, and earth. 

However, the context information provided does not include the recommended directions for placing a stove or a bed. To ensure positive energy and well-being for these objects, you may need to consult additional sources or guidelines that specifically address the placement of stoves and beds within the principles of feng shui or similar systems of spatial arrangement.

Final Response: The recommended directions for placing an aquarium to promote positive energy and well-being in your home are North-West and South-East. It is important to note that you should avoid placing an aquarium in the kitchen, bedroom, bathroom, or the center of the home, as these locations are considered inauspicious due to the conflicting elements of fire, water, and earth.

However, the context information provided does not include the recommended directions for placing a stove or a bed. To ensure positive energy and well-being for these objects, you may need to consult additional sources or guidelines that specifically address the placement of stoves and beds within the principles of feng shui or similar systems of spatial arrangement.

## Observation (Between GPT 4  & GPT 4.0-1106-preview)

There is indeed a difference between the two models. The GPT-4.0-1106-preview give a more clear and comprehensive response than the gpt-4 models

## Checking whether the GPT-3.5-turbo-1106 (Fine Tune) or GPT-4-1106-preview works better

## Generate Training Data

Here, we use GPT-3.5-turbo and the `OpenAIFineTuningHandler` to collect data that we want to train on.

In [7]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0),
    context_window=2048,  # limit the context window artifically to test refine process
    callback_manager=callback_manager,
)
#GPT-3.5-turbo model to understand the semantic content of documents and then uses this understanding to find documents that are semantically similar to a given query

In [8]:
questions = []
with open("modified_train_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())
#reads questions from a file named modified_train_questions.txt and stores them in a list named questions.

In [9]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(docs, service_context=gpt_4_context)

query_engine = index.as_query_engine(similarity_top_k=2)
#create a VectorStoreIndex from a collection of documents (docs) using the LlamaIndex library, and then convert this index into a query engine 

In [10]:
for question in questions:
    response = query_engine.query(question)

# loop that iterates over a list of questions, querying a query_engine for each question and storing the response in a variable named respons

## Create Fine Tuned Engine



In [11]:
finetuning_handler.save_finetuning_events("finetune.jsonl")
#save fine-tuning events to a JSONL file called finetune.

Wrote 25 examples to finetune.jsonl


## Evaluating Fine Tuned Engine

After some time, your model will be done training!

The next step is running our fine-tuned model on our eval dataset again to measure any performance increase.

In [12]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())
#reads a text file named eval_questions.txt line by line, strips any leading or trailing whitespace (including newlines) from each line, and appends each line to a list named questions

In [13]:
from llama_index import VectorStoreIndex

ft_context = ServiceContext.from_defaults(
    llm=OpenAI(model="ft:gpt-3.5-turbo-1106:personal:fengshui:9IzbaIn8",temperature=0), context_window=2048
)
index = VectorStoreIndex.from_documents(docs, service_context=ft_context)

query_engine = index.as_query_engine(similarity_top_k=2)

#query engine using the VectorStoreIndex from the llama_index library, specifically tailored for fine-tuning a language model (LLM) like GPT-3.5 Turbo. 

In [14]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))
#query engine to process a list of questions and collect both the contexts (source nodes) and answers from the responses.

In [15]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [answer_relevancy, faithfulness])
print(result)
#evaluate result on answer_relevancy & faithfulness using ragas. Ragas score  = (answer_relevancy + faithfulness) / 2

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

{'answer_relevancy': 0.9385, 'faithfulness': 0.9767}


| Model                            | Ragas Score | Answer Relevancy   | Faithfulness   |
|:----------------:                |:-----------:|:------------------:|:--------------:|
| GPT-4                            | 0.9358      | 0.8894             | 0.9822         |
| GPT-4.0-1106-preview             | 0.9575      | 0.9266             | 0.9840         |
| GPT-3.5-Turbo-1106 (Fine Tune)   | 0.9576      | 0.9385             | 0.9767         |
{'ragas_score': 0.9358, 'answer_relevancy': 0.8894, 'faithfulness': 0.9822}
{'ragas_score': 0.9575, 'answer_relevancy': 0.9329, 'faithfulness': 0.9822}
{'ragas_score': 0.9576, 'answer_relevancy': 0.9385, 'faithfulness': 0.9767}

Date: 28 April 2024

## Observation between GPT-4-1106-preview vs GPT-3.5-turbo-1106 (Fine Tune) 

In [37]:
print(questions[1])  
#access the second index/element in the questions

What are the recommended directions for placing a stove, aquarium, and bed to promote positive energy and well-being in my home?


What are the recommended directions for placing a stove, aquarium, and bed to promote positive energy and well-being in my home?


In [60]:
ft_context = ServiceContext.from_defaults(
    #llm=ft_llm,
    llm=OpenAI(model="gpt-4-1106-preview",temperature=0),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [61]:
query_engine = index.as_query_engine(service_context=ft_context)

response = query_engine.query(questions[1])

display_response(response)
#iterative, allowing for the evaluation of different embeddings and querying strategies.

**`Final Response:`** The recommended direction for placing an aquarium to promote positive energy and well-being in your home is either the North-West or South-East direction. It is important to note that you should avoid placing an aquarium in the kitchen, bedroom, bathroom, or the center of the home, as these locations are considered inauspicious due to the conflicting elements of fire, water, and earth. The information provided does not include recommended directions for placing a stove or a bed, so I cannot advise on those objects.

Final Response: The recommended direction for placing an aquarium to promote positive energy and well-being in your home is either the North-West or South-East direction. It is important to note that you should avoid placing an aquarium in the kitchen, bedroom, bathroom, or the center of the home, as these locations are considered inauspicious due to the conflicting elements of fire, water, and earth. The information provided does not include recommended directions for placing a stove or a bed, so I cannot advise on those objects.

In [62]:
ft_context = ServiceContext.from_defaults(
    #llm=ft_llm,
    llm=OpenAI(model="ft:gpt-3.5-turbo-1106:personal:fengshui:9IzbaIn8",temperature=0),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [63]:
query_engine = index.as_query_engine(service_context=ft_context)

response = query_engine.query(questions[1])

display_response(response)
#iterative, allowing for the evaluation of different embeddings and querying strategies.

**`Final Response:`** The recommended directions for placing a stove is South-East, for an aquarium is North-West, and for a bed is South-West to promote positive energy and well-being in your home.

Final Response: The recommended directions for placing a stove is South-East, for an aquarium is North-West, and for a bed is South-West to promote positive energy and well-being in your home.

## Conclusion : The GPT-3.5-turbo-1106 fine-tuned model achieved a RAGAS score of 0.9576, slightly surpassing the GPT-4-1106-preview model's score of 0.9575. Despite the GPT-4-1106-preview model providing more detailed responses, it failed to address all queries regarding the three objects in question. Conversely, the GPT-3.5-turbo-1106 model successfully answered all queries, meeting the basic requirements expected of a chatbot. Given its cost-effectiveness, higher RAGAS score, and comprehensive response capabilities, the GPT-3.5-turbo-1106 fine-tuned model has been selected for this capstone project.
